In [ ]:
import os
import requests
import zipfile
import sqlite3
import glob

### Creating a directory called Staging

In [ ]:
# Creating an object called staging_dir_name as pointer to Staging
staging_dir_name = "staging4"

In [ ]:
# Creating a directory in a local folder
os.mkdir(staging_dir_name)

In [ ]:
# Creating a pointer to the given source 
url="https://data.medicare.gov/views/bg9k-emty/files/0a9879e0-3312-4719-a1db-39fd114890f1?content_type=application%2Fzip%3B%20charset%3Dbinary&filename=Hospital_Revised_Flatfiles.zip"

In [ ]:
# Loading the data from URL into new object called r
r = requests.get(url)

In [ ]:
# To know the size of the file in Content-Lenth
r.headers

In [ ]:
#Creating a pointer to zip file  
zip_file_name=os.path.join(staging_dir_name, "Hospital_Revised_Flatfiles.zip")

In [ ]:
# Creating a zip file with write binary format
zf = open(zip_file_name, "wb")

In [ ]:
# Writing the content of URL into the pointer to zip file
zf.write(r.content)

In [ ]:
zf.close()

### Unzipping the File

In [ ]:
z = zipfile.ZipFile(zip_file_name,"r")

In [ ]:
z.extractall(staging_dir_name)

### Creating a SQLite database in local directory

In [ ]:
conn = sqlite3.connect('medicare_hospital_compare3.db')

In [ ]:
conn

### Creating a table for each CSV file

In [ ]:
def do_directory(staging_dir_name, conn):
    print("a")
    for filename in glob.glob(os.path.join(staging_dir_name, '*.csv')):
        do_file(filename, conn)
        
def do_file(filename, conn):
        with open(filename) as f:
            with conn:
                data = csv.DicReader(f)
                cols = data.fieldnames
                table=os.path.splitext(os.path.basename(filename))[0]

                sql = 'drop table if exists "{}"'.format(table)
                db.execute(sql)

                sql = 'create table "{table}" ( {cols} )'.format(
                    table=table,
                    cols=','.join('"{}"'.format(col) for col in cols))
                db.execute(sql)

                sql = 'insert into "{table}" values ( {vals} )'.format(
                    table=table,
                    vals=','.join('?' for col in cols))
                db.executemany(sql, (list(map(row.get, cols)) for row in data))

if __name__ == '__main__':
    conn = sqlite3.connect('medicare_hospital_compare3.db')
    do_directory('.', conn)